# Training
Training process of the final model with full dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pretrainedmodels
import pandas as pd
import os

from tools import *

def write_log(logfile, train_loss, test_loss, test_score, lr):
    with open(logfile, "a+") as log:
        log.write("{}\t{}\t{}\t{}\n".format(train_loss, test_loss, test_score, lr))


In [2]:
device = get_device()

Setting DEVICE:
	 MPS is available


## Importing label dataset

To run the full crossvalidation, remove .head(30000). This will multiply about by 20 the time required by this notebook.

In [3]:

# FOLDERS PATH
source_dir = 'histopathologic-cancer-detection/'
train_im_source_dir = source_dir+'/train'

# IMPORTING DATA
train_data = pd.read_csv(os.path.join(source_dir,'train_labels.csv')) 
train_data = train_data.sample(frac=1, random_state=10)



In [4]:
data_ids = list(train_data.id)
data_labels = list(train_data.label)

## Training

In [5]:
# Logging directories
model_source_dir = 'trained_model/'
model_name = 'final'

# Batch size
b_size = int(96) # batch size

# Epochs
n_epochs = 15


### Training loop

In [6]:

# Setting log files
logfile =  model_source_dir+'/{}.logfile.txt'.format(model_name)
best_w_path = model_source_dir+'/{}.best.pt'.format(model_name)
model_path =  model_source_dir+'/{}.model.pt'.format(model_name)
    
# Data augmentation functions
training_aug = aug_train() 
    
# Learning Rate setting. This will be modified according to cyclic scheduler
curr_lr = 3e-3 
    
# Loader for the training and the validation
train_loader = torch.utils.data.DataLoader(DataGenerator(
                                                data_ids,
                                                data_labels,
                                                training_aug, 
                                                train_im_source_dir),
                                            pin_memory=False,
                                            num_workers=4,
                                            batch_size=b_size) 
                                            
# Loss function 
loss_f = nn.BCELoss() # BINARY CROSS ENTROPY

# Import pretrained model
base_model = pretrainedmodels.resnet34(num_classes=1000, 
                                           pretrained='imagenet').to(device) 

    
# Shape the model    
model = Net(base_model, 512).to(device)

# Optimizer
# Some layers are freezed for the first iteration, by setting the learning rate to zero
optimizer = optim.SGD([{'params': model.layer0.parameters(), 'lr': 0},
                        {'params': model.layer2.parameters(), 'lr': 0},
                        {'params': model.layer1.parameters(), 'lr': 0},
                        {'params': model.layer3.parameters(), 'lr': 0},
                        {'params': model.layer4.parameters(), 'lr': 0},
                        {'params': model.classif.parameters()}], lr=0.05, momentum=0.9)
    
# First Training procedure
train_loss = train(model= model,
                    train_loader= train_loader, 
                    optimizer= optimizer, 
                    epoch= 0, 
                    log_interval= 100, 
                    loss_f= loss_f, 
                    scheduler= None,
                    device=device)
    
    
# Loop on epochs

# Values to monitor the loss trough the loop on epochs
best_score = 0
best_loss = 1e5
idx_stop = 0

for epoch in range(n_epochs):

    # Print current epoch
    print(f'Begin of epoch {epoch}')

    # Optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=curr_lr, momentum=0.9)

    # Scheduler for triangular cyclic learning rate
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=curr_lr, max_lr=3*curr_lr, mode = 'triangular')

    # Train procedure
    train_loss = train(model= model,
                    train_loader= train_loader, 
                    optimizer= optimizer, 
                    epoch= epoch, 
                    log_interval= 1000, 
                    loss_f= loss_f, 
                    scheduler= scheduler,
                    device = device)
    
    print('\t end epoch - loss: {:.6f}'.format(train_loss))
        
        
    
torch.save(model.state_dict(), model_path)



/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train Epoch: 0 [0] 	Loss: 0.711788
Train Epoch: 0 [9600] 	Loss: 0.575053
Train Epoch: 0 [19200] 	Loss: 0.516960
Train Epoch: 0 [28800] 	Loss: 0.508497
Train Epoch: 0 [38400] 	Loss: 0.508654
Train Epoch: 0 [48000] 	Loss: 0.495767
Train Epoch: 0 [57600] 	Loss: 0.499734
Train Epoch: 0 [67200] 	Loss: 0.491936
Train Epoch: 0 [76800] 	Loss: 0.502565
Train Epoch: 0 [86400] 	Loss: 0.492402
Train Epoch: 0 [96000] 	Loss: 0.495736
Train Epoch: 0 [105600] 	Loss: 0.494938
Train Epoch: 0 [115200] 	Loss: 0.490527
Train Epoch: 0 [124800] 	Loss: 0.491509
Train Epoch: 0 [134400] 	Loss: 0.487060
Train Epoch: 0 [144000] 	Loss: 0.500588
Train Epoch: 0 [153600] 	Loss: 0.493360
Train Epoch: 0 [163200] 	Loss: 0.486640
Train Epoch: 0 [172800] 	Loss: 0.494440
Train Epoch: 0 [182400] 	Loss: 0.490889
Train Epoch: 0 [192000] 	Loss: 0.493751
Train Epoch: 0 [201600] 	Loss: 0.490738
Train Epoch: 0 [211200] 	Loss: 0.477821
Mean train loss on epoch 0 : 0.5083197356695714
Begin of epoch 0
Train Epoch: 0 [0] 	Loss: 0.491